In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import numpy as np
import itertools

import time

In [310]:
url2048 = 'https://play2048.co/'

class Session:
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.driver.get(url2048)
        self.current_grid = self.get_tiles_grid()
    
    def update_grid(self):
        self.current_grid = self.get_tiles_grid()

    def end_session(self):
        """closes browser."""
        self.driver.close()

    def restart_game(self):
        restart_btn = self.driver.find_element('class name', 'restart-button')
        restart_btn.click()
        self.update_grid()

    def right(self):
        """Move right."""
        action = ActionChains(self.driver)
        action.key_down(Keys.ARROW_RIGHT)
        action.perform()
        self.update_grid()

    def left(self):
        """Move left."""
        action = ActionChains(self.driver)
        action.key_down(Keys.ARROW_LEFT)
        action.perform()
        self.update_grid()

    def up(self):
        """Move up."""
        action = ActionChains(self.driver)
        action.key_down(Keys.ARROW_UP)
        action.perform()
        self.update_grid()

    def down(self):
        """Move down."""
        action = ActionChains(self.driver)
        action.key_down(Keys.ARROW_DOWN)
        action.perform()
        self.update_grid()

    def get_board(self):
        """Returns a list of web elements of 'tiles-state' on board"""
        container = self.driver.find_element('class name', 'tile-container')
        return container.find_elements('xpath', '*')

    def get_tiles_grid(self):
        """Returns np.array with values of tiles."""
        tiles_grid = np.array([[0 for _ in range(4)] for _ in range(4)])
        board = self.get_board()
        for tile in board:
            tile_desc = tile.get_attribute('class').split(' tile-')
            position = (int(tile_desc[2].split('-')[1]) - 1,
                        int(tile_desc[2].split('-')[2]) - 1)
            tile_value = int(tile_desc[1])
            tiles_grid[position] = tile_value
        return np.transpose(tiles_grid)
    

In [477]:
def move_row(row):
    """rearrange"""
    temp = [x for x in row if x != 0]
    zeros = len(row) - len(temp)
    i = 1
    while i < len(temp):
        if temp[-i] == temp[-i-1]:
            temp[-i] = temp[-i] * 2
            del temp[-i-1]
            zeros += 1
        i += 1
    return [0] * zeros + temp




def get_board_if_move(cur_grid, direction):
    'Return board grid if moved in direction (without added tile).'
    grid = np.copy(cur_grid)
    
    if direction == 'down':
        for i in range(4):
            rearranged_col = move_row(grid[:, i])
            grid[:, i] = rearranged_col

    elif direction == 'up':
        for i in range(4):
            rearranged_col = move_row(np.flip(grid[:, i]))
            rearranged_col.reverse()
            grid[:, i] = rearranged_col

    elif direction == 'left':
        for i in range(4):
            rearranged_row = move_row(np.flip(grid[i, :]))
            rearranged_row.reverse()
            grid[i, :] = rearranged_row

    elif direction == 'right':
        for i in range(4):
            rearranged_row = move_row(grid[i, :])
            grid[i, :] = rearranged_row

    return grid


def get_max_tile(grid):
    """Returns max tile value and a list of positions."""
    value = np.amax(grid)
    row, col = np.where(grid==value)
    max_coor = list(zip(row, col))
    return value, max_coor


def are_neighbors(tiles_pos):
    """Gets a list of positions, returns True if a couple are neighbors."""
    for i, pos in enumerate(tiles_pos):
        for j in range(i+1, len(tiles_pos)):
            if ((pos[0] == tiles_pos[j][0] and
                abs(pos[1] - tiles_pos[j][1]) == 1) or
                (pos[1] == tiles_pos[j][1] and 
                abs(pos[0] - tiles_pos[j][0]) == 1)):
                return True
    return False


def is_higher_max_value(grid1, grid2):
    """Returns True if grid2's max value is bigger than grid1's."""
    max_value_1 = get_max_tile(grid1)[0]
    max_value_2 = get_max_tile(grid2)[0]
    if max_value_1 > max_value_2:
        return True
    return False

In [468]:
new_session = Session()
# new_session.update_grid()
new_session.current_grid

array([[0, 0, 0, 2],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 2, 0, 0]])

In [474]:
# for _ in range(3):
new_session.right()
new_session.left()
# new_session.up()
new_session.down()
# new_session.current_grid

In [384]:
print(new_session.current_grid)
# print('----predicted----')
# print(get_board_if_move(new_session.current_grid, 'up'))

[[4 4 8 0]
 [2 2 0 0]
 [0 0 0 0]
 [2 0 0 0]]


In [402]:
a = np.copy(get_board_if_move(new_session.current_grid, 'up'))
print(type(a))
print('---')
# new_session.up()
# print(new_session.current_grid - a)
print('---')
print(type(new_session.current_grid))

<class 'numpy.ndarray'>
---
---
<class 'numpy.ndarray'>


In [462]:
a

array([[ 1,  1, 31,  1],
       [ 3,  2, 15,  3],
       [ 1, 15,  6, -1],
       [-1, -1, -1,  1]])

In [464]:
b = np.array([[2,  2,  3,  2],
              [4,  3,  16,  4],
              [2,  16,  7,  0],
              [0,  0,  0,  2]])

# val, pos = get_max_tile(b)

# are_neighbors(pos)
higher_or_equal_max_value(a,b)

1

In [478]:
moves = dict()
cur_grid = np.array(new_session.current_grid)
moves['up'] = is_higher_max_value(cur_grid, get_board_if_move(cur_grid, 'up'))
moves['down'] = is_higher_max_value(cur_grid, get_board_if_move(cur_grid, 'down'))
moves['right'] = is_higher_max_value(cur_grid, get_board_if_move(cur_grid, 'right'))
moves['left'] = is_higher_max_value(cur_grid, get_board_if_move(cur_grid, 'left'))
moves

{'up': False, 'down': False, 'right': False, 'left': False}